In [5]:
'''
import numpy as np
#import urllib2
import pdb
#response = urllib2.urlopen("http://www.relianceiccrankings.com/datespecific/odi/?stattype=batting&day=24&month=01&year=2000")
#page_source = response.read()
import requests
username = 'vasus'
password = ''
url = 'http://www.relianceiccrankings.com/datespecific/odi/?stattype=batting&day=24&month=01&year=2000'
#r = requests.get(url, auth=(username, password))  
r = requests.get(url, verify=False)
page = r.content
'''


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import scipy.stats as sts

from bs4 import BeautifulSoup

headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'Custom User Agent 1.0',
        'From': 'shubham9@illinois.edu'  # Stick in your IITK email id
    }
)

all_data = {}

In [11]:
import csv
ids=[]
with open('../../../../Downloads/ScrapyIMDB/scrapyIMDB/data/movie_list.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if row[-1][-1].isnumeric():
            ids.append(row[-1])
# print(ids)
#             print(row[-1])
#         else:
#             print(row[-1])
#             print('no')

In [13]:
# ids=['tt2488496']
for k in ids:
    url = 'http://www.imdb.com/title/'+str(k)+'/reviews'
    sort =['?sort=helpfulnessScore','?sort=submissionDate','?sort=totalVotes','?sort=reviewVolume','?sort=userRating']
    direc =['&dir=desc','&dir=asc']
    data=[]
    user=[]
    for i in sort:
        for j in direc:
            r = requests.get(url + i+j, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            listing = soup.find(class_='lister')
#             print(url + i+j)
            reviews = listing.find_all(class_='lister-item-content')
            for row in reviews:
                if str(row.find(class_='display-name-date')).split('tt_urv">')[1].split('</')[0] in user:
                    break
                else:
                    user.append(str(row.find(class_='display-name-date')).split('tt_urv">')[1].split('</')[0])
                if str(row).find('text show-more__control')>=0:
                    text =str(row.find(class_='text show-more__control')).split('<div class="text show-more__control">')[1].split('</div>')[0]
                else:
                    print('doesnt exist')
                    text=''
                if str(row.find(class_='ipl-ratings-bar')).find('<span>') >=0:
                    rating = int(str(row.find(class_='ipl-ratings-bar')).split('<span>')[1].split('</span>')[0])
                else:
                    rating=-1
                    print('not')
                title = str(row.find(class_='title')).split('>')[1].split('<')[0]
                data.append([rating,title,text])
    print(len(data))
    all_data[k]=data

http://www.imdb.com/title/tt2488496/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt2488496/reviews?sort=helpfulnessScore&dir=asc
not
not
http://www.imdb.com/title/tt2488496/reviews?sort=submissionDate&dir=desc
not
not
not
not
not
not
not
http://www.imdb.com/title/tt2488496/reviews?sort=submissionDate&dir=asc
not
http://www.imdb.com/title/tt2488496/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt2488496/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt2488496/reviews?sort=reviewVolume&dir=desc
not
not
not
not
http://www.imdb.com/title/tt2488496/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt2488496/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt2488496/reviews?sort=userRating&dir=asc
197
http://www.imdb.com/title/tt3659388/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt3659388/reviews?sort=helpfulnessScore&dir=asc
not
not
http://www.imdb.com/title/tt3659388/reviews?sort=submissionDate&di

http://www.imdb.com/title/tt1823672/reviews?sort=submissionDate&dir=desc
not
http://www.imdb.com/title/tt1823672/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt1823672/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt1823672/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt1823672/reviews?sort=reviewVolume&dir=desc
not
not
not
http://www.imdb.com/title/tt1823672/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt1823672/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt1823672/reviews?sort=userRating&dir=asc
117
http://www.imdb.com/title/tt2908446/reviews?sort=helpfulnessScore&dir=desc
http://www.imdb.com/title/tt2908446/reviews?sort=helpfulnessScore&dir=asc
not
not
http://www.imdb.com/title/tt2908446/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt2908446/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt2908446/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt2908446/reviews?sort=

http://www.imdb.com/title/tt1843866/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt1843866/reviews?sort=reviewVolume&dir=desc
http://www.imdb.com/title/tt1843866/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt1843866/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt1843866/reviews?sort=userRating&dir=asc
105
http://www.imdb.com/title/tt2084970/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt2084970/reviews?sort=helpfulnessScore&dir=asc
not
not
not
http://www.imdb.com/title/tt2084970/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt2084970/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt2084970/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt2084970/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt2084970/reviews?sort=reviewVolume&dir=desc
not
not
http://www.imdb.com/title/tt2084970/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt2084970/reviews?sort=userR

http://www.imdb.com/title/tt2911666/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt2911666/reviews?sort=userRating&dir=desc
not
http://www.imdb.com/title/tt2911666/reviews?sort=userRating&dir=asc
100
http://www.imdb.com/title/tt2582846/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt2582846/reviews?sort=helpfulnessScore&dir=asc
not
http://www.imdb.com/title/tt2582846/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt2582846/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt2582846/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt2582846/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt2582846/reviews?sort=reviewVolume&dir=desc
http://www.imdb.com/title/tt2582846/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt2582846/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt2582846/reviews?sort=userRating&dir=asc
80
http://www.imdb.com/title/tt1490017/reviews?sort=helpfulnessScore

http://www.imdb.com/title/tt2024544/reviews?sort=userRating&dir=asc
117
http://www.imdb.com/title/tt1981115/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt1981115/reviews?sort=helpfulnessScore&dir=asc
http://www.imdb.com/title/tt1981115/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt1981115/reviews?sort=submissionDate&dir=asc
not
http://www.imdb.com/title/tt1981115/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt1981115/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt1981115/reviews?sort=reviewVolume&dir=desc
not
http://www.imdb.com/title/tt1981115/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt1981115/reviews?sort=userRating&dir=desc
not
http://www.imdb.com/title/tt1981115/reviews?sort=userRating&dir=asc
96
http://www.imdb.com/title/tt1392214/reviews?sort=helpfulnessScore&dir=desc
http://www.imdb.com/title/tt1392214/reviews?sort=helpfulnessScore&dir=asc
http://www.imdb.com/title/tt1392214/reviews?sort=su

...

http://www.imdb.com/title/tt0080684/reviews?sort=submissionDate&dir=desc
not
not
not
http://www.imdb.com/title/tt0080684/reviews?sort=submissionDate&dir=asc
not
not
not
not
not
not
not
not
not
http://www.imdb.com/title/tt0080684/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt0080684/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt0080684/reviews?sort=reviewVolume&dir=desc
not
http://www.imdb.com/title/tt0080684/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt0080684/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt0080684/reviews?sort=userRating&dir=asc
112
http://www.imdb.com/title/tt0993846/reviews?sort=helpfulnessScore&dir=desc
not
not
not
not
not
http://www.imdb.com/title/tt0993846/reviews?sort=helpfulnessScore&dir=asc
not
http://www.imdb.com/title/tt0993846/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt0993846/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt0993846/reviews?sort=totalVotes&dir=de

http://www.imdb.com/title/tt0167260/reviews?sort=helpfulnessScore&dir=asc
not
http://www.imdb.com/title/tt0167260/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt0167260/reviews?sort=submissionDate&dir=asc
not
not
not
not
http://www.imdb.com/title/tt0167260/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt0167260/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt0167260/reviews?sort=reviewVolume&dir=desc
http://www.imdb.com/title/tt0167260/reviews?sort=reviewVolume&dir=asc
not
http://www.imdb.com/title/tt0167260/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt0167260/reviews?sort=userRating&dir=asc
141
http://www.imdb.com/title/tt1345836/reviews?sort=helpfulnessScore&dir=desc
http://www.imdb.com/title/tt1345836/reviews?sort=helpfulnessScore&dir=asc
not
not
not
not
http://www.imdb.com/title/tt1345836/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt1345836/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt

http://www.imdb.com/title/tt1130884/reviews?sort=helpfulnessScore&dir=desc
http://www.imdb.com/title/tt1130884/reviews?sort=helpfulnessScore&dir=asc
not
http://www.imdb.com/title/tt1130884/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt1130884/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt1130884/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt1130884/reviews?sort=totalVotes&dir=asc
not
http://www.imdb.com/title/tt1130884/reviews?sort=reviewVolume&dir=desc
not
not
not
not
http://www.imdb.com/title/tt1130884/reviews?sort=reviewVolume&dir=asc
not
http://www.imdb.com/title/tt1130884/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt1130884/reviews?sort=userRating&dir=asc
107
http://www.imdb.com/title/tt0071562/reviews?sort=helpfulnessScore&dir=desc
not
not
not
http://www.imdb.com/title/tt0071562/reviews?sort=helpfulnessScore&dir=asc
not
http://www.imdb.com/title/tt0071562/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/t

http://www.imdb.com/title/tt1590193/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt1590193/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt1590193/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt1590193/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt1590193/reviews?sort=reviewVolume&dir=desc
http://www.imdb.com/title/tt1590193/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt1590193/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt1590193/reviews?sort=userRating&dir=asc
42
http://www.imdb.com/title/tt1464763/reviews?sort=helpfulnessScore&dir=desc
http://www.imdb.com/title/tt1464763/reviews?sort=helpfulnessScore&dir=asc
http://www.imdb.com/title/tt1464763/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt1464763/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt1464763/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt1464763/reviews?sort=totalVotes&dir=asc
http:/

http://www.imdb.com/title/tt6237966/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt6237966/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt6237966/reviews?sort=userRating&dir=asc
19
http://www.imdb.com/title/tt7153766/reviews?sort=helpfulnessScore&dir=desc
not
http://www.imdb.com/title/tt7153766/reviews?sort=helpfulnessScore&dir=asc
http://www.imdb.com/title/tt7153766/reviews?sort=submissionDate&dir=desc
http://www.imdb.com/title/tt7153766/reviews?sort=submissionDate&dir=asc
http://www.imdb.com/title/tt7153766/reviews?sort=totalVotes&dir=desc
http://www.imdb.com/title/tt7153766/reviews?sort=totalVotes&dir=asc
http://www.imdb.com/title/tt7153766/reviews?sort=reviewVolume&dir=desc
http://www.imdb.com/title/tt7153766/reviews?sort=reviewVolume&dir=asc
http://www.imdb.com/title/tt7153766/reviews?sort=userRating&dir=desc
http://www.imdb.com/title/tt7153766/reviews?sort=userRating&dir=asc
45
http://www.imdb.com/title/tt6684714/reviews?sort=helpfulnessScore&dir=desc

In [14]:
# print(all_data)
import pickle
pickle.dump(all_data,open( "alldata.data", "wb" ) )

In [15]:
for i in ids:
    print(all_data[k])
    break

[[6, "'What was the point?'", "An '80s drive-in style shoot-em-up revenge formula that once would have been a vehicle for Charles Bronson and Jan-Michael Vincent in their later days.\nWas not expecting to get through it, but better production quality than I expected and decent 'better than the script deserved' acting made it an almost painless diversion, although the script/logic is too drive-in shallow to allow the attempted emotional triggers to fire and, thus, recreates one of life's difficult/interesting questions:\n'What was the point?'"], [3, 'Contrived and predictable', '"Acts of Violence" suffers from a really poorly written script that relies heavily on tropes and plot devices borrowed from other films, including "Taken" and "Casino Royale." The villains are into everything, kidnapping, human trafficking, prostitution and drugs, including selling narcotics intended for use on rhinos. Their motivations seem to be to do whatever might make a visually interesting scene. It\'s lar